In [2]:
pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=f68f6cdeeac71eaf1c0ba62b656d0ab663fb77d621cbb8b7143a0df6fc63e1f7
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=72d05e85837261b4f198268e0d669559b9191aaf8a45154321a4890af24b9dcd
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pyLDAvis sklearn


In [16]:
import sys
import re, numpy as np, pandas as pd
from pprint import pprint
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])
%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import requests 
import time 
import bs4
from bs4 import BeautifulSoup
import random
from gensim import corpora, models, similarities
import sys

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
fStar = pd.read_csv('/content/fiveStarReviews.csv')
oStar = pd.read_csv('/content/oneStarReviews.csv')

In [15]:
from pandas.core.arrays.boolean import BooleanDtype

def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', str(sent))  
        sent = re.sub('\s+', ' ', str(sent))  
        sent = re.sub("\'", "", str(sent))  
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  
        
def cleaner(df):
  data = df.text.values.tolist()
  data_words = list(sent_to_words(data))
  return data_words, data
print('*')
print('**')
print('***')
print('Enter high or low, to analyze 5 star vs 1 star reviews')
print('Enter the number of topics you would like to analyze (rec: 5-10)')
print('***')
print('**')
print('*')

def topicMaster(target,num_topics, savedAs):
  if target.lower() == 'high':
    target = fStar
  elif target.lower() == 'low':
    target = oStar
  else:
    target = fStar
    print('high/low not entered, default: 5 star reviews')
  data_words, data = cleaner(target)
  bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
  trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)

  def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
      """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
      texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
      texts = [bigram_mod[doc] for doc in texts]
      texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
      texts_out = []
      nlp = spacy.load('en_core_web_sm')
      for sent in texts:
          doc = nlp(" ".join(sent)) 
          texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
      texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
      return texts_out
  data_ready = process_words(data_words)
  id2word = corpora.Dictionary(data_ready)

  corpus = [id2word.doc2bow(text) for text in data_ready]
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=num_topics, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=10,
                                            passes=10,
                                            alpha='symmetric',
                                            iterations=100,
                                            per_word_topics=True)

  pprint(lda_model.print_topics())

  def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
      sent_topics_df = pd.DataFrame()
      for i, row_list in enumerate(ldamodel[corpus]):
          row = row_list[0] if ldamodel.per_word_topics else row_list            
          row = sorted(row, key=lambda x: (x[1]), reverse=True)
          for j, (topic_num, prop_topic) in enumerate(row):
              if j == 0: 
                  wp = ldamodel.show_topic(topic_num)
                  topic_keywords = ", ".join([word for word, prop in wp])
                  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
              else:
                  break
      sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
      sent_topics_df=sent_topics_df[sent_topics_df['Dominant_Topic']>=.5]
      contents = pd.Series(texts)
      sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
      return(sent_topics_df)

  df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)
  df_dominant_topic = df_topic_sents_keywords.reset_index()
  df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

  pd.options.display.max_colwidth = 100
  sent_topics_sorteddf_mallet = pd.DataFrame()
  sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')
  for i, grp in sent_topics_outdf_grpd:
      sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                              grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                              axis=0)
  sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)
  sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

  import pyLDAvis
  import pyLDAvis.gensim_models
  pyLDAvis.enable_notebook()
  vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary=lda_model.id2word)
  p = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary=lda_model.id2word)
  pyLDAvis.save_html(p, savedAs)
  return vis

topicMaster(input(),int(input()), 'fStar.html')

<>:5: DeprecationWarning: invalid escape sequence \S
<>:6: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:6: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:6: DeprecationWarning: invalid escape sequence \s
<ipython-input-15-f59041caf8b6>:5: DeprecationWarning: invalid escape sequence \S
  sent = re.sub('\S*@\S*\s?', '', str(sent))
<ipython-input-15-f59041caf8b6>:6: DeprecationWarning: invalid escape sequence \s
  sent = re.sub('\s+', ' ', str(sent))


*
**
***
Enter high or low, to analyze 5 star vs 1 star reviews
Enter the number of topics you would like to analyze (rec: 5-10)
***
**
*
low
5


/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: Depr

[(0,
  '0.065*"amazon" + 0.047*"disappoint" + 0.035*"bottle" + 0.035*"order" + '
  '0.025*"jump" + 0.021*"definitely" + 0.021*"recommend" + 0.021*"survive" + '
  '0.021*"spring" + 0.021*"expect"'),
 (1,
  '0.069*"generic" + 0.059*"xyzal" + 0.055*"feel" + 0.051*"version" + '
  '0.031*"back" + 0.030*"ingredient" + 0.026*"really" + 0.024*"less" + '
  '0.021*"compare" + 0.019*"day"'),
 (2,
  '0.045*"pill" + 0.040*"effective" + 0.036*"eye" + 0.033*"absolutely" + '
  '0.032*"however" + 0.029*"congest" + 0.029*"son" + 0.022*"morning" + '
  '0.022*"experience" + 0.021*"medication"'),
 (3,
  '0.044*"relief" + 0.039*"still" + 0.032*"allergy" + 0.031*"stick" + '
  '0.026*"week" + 0.022*"reason" + 0.022*"month" + 0.022*"night" + '
  '0.021*"case" + 0.020*"suffer"'),
 (4,
  '0.157*"work" + 0.083*"brand" + 0.061*"cheap" + 0.038*"well" + '
  '0.031*"usually" + 0.030*"name" + 0.029*"decide" + 0.026*"option" + '
  '0.021*"great" + 0.021*"stop"')]


/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.045799 -0.180563       1        1  24.437127
4     -0.215846  0.007213       2        1  21.212132
2      0.063441  0.057672       3        1  20.674828
3      0.052790  0.056162       4        1  18.983820
0      0.053816  0.059516       5        1  14.692093, topic_info=              Term       Freq      Total Category  logprob  loglift
6             work  13.000000  13.000000  Default  30.0000  30.0000
3            brand   7.000000   7.000000  Default  29.0000  29.0000
74         generic   6.000000   6.000000  Default  28.0000  28.0000
17          amazon   4.000000   4.000000  Default  27.0000  27.0000
21           cheap   5.000000   5.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
115         matter   0.475509   1.053082   Topic5  -4.7471   1.1228
89          charge   0.392859   0.970506   Topic5  -4.9380   1.0135
55   unfortunately   0.318691   0.908427   Topic5  -5.1472   0.8704
2            price   0.964435   2.766146   Topic5  -4.0399   0.8642
113            buy   0.475093   1.491264   Topic5  -4.7480   0.7740

[184 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
31        3  0.979467  absolutely
85        1  0.938097         ago
134       4  0.707640     allergy
169       2  0.756153      almost
17        5  0.964412      amazon
...     ...       ...         ...
6         2  0.922565        work
167       1  0.841536       xyzal
76        1  0.941103        year
71        1  0.533580      zyrtec
126       4  0.636171       zyzal

[147 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 3, 4, 1])